In [1]:
import json
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
from retry import retry
config = dotenv_values('.env')

In [2]:
MODEL_NAME = 'qwen-max'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [3]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [4]:
def get_question_prompt(data):
    prompt = f"""你是一个逻辑推理问题出题专家，以下json是一个problem，对应一些子问题。除了这些子问题外再生成三个其他子问题，保持格式一致并给出参考答案。
## problem如下
{data}
## 输出格式如下
'''json{{
promble:''
questions:[
question:''
options:[]
]
}}...
"""
    return prompt


In [5]:
def read_file(ifn):
    # 读取输入文件
    data = []
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    return data

In [6]:
def write_jsonl(data):
    with open('data/output.jsonl', 'w', encoding='utf-8') as file:
        for item in data:
            json.dump(item, file)
            file.write('\n')

In [13]:
def process_data(data):
    query = get_question_prompt(data)
    try:
        respond = call_qwen_api(MODEL_NAME, query)
        # 清理 JSON 字符串
        if respond.startswith('```json'):
            # 如果有前缀，则清理
            respond = respond.replace('```json', '', 1).replace('```', '')
        # 解析 JSON 字符串
        data = json.loads(respond)
    except Exception as e:
        print(data)
        print(f"Error: {str(e)}")
    return data

In [11]:
file = read_file('data/round1_train_data.jsonl')

In [12]:
from tqdm import tqdm
import concurrent.futures
file = file[:5]

# 并行批量处理
def batch_process_questions(file, max_workers=5):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_data, data) for data in file]
        results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(file))]
    return results

results = batch_process_questions(file)

write_jsonl(results)

 80%|████████  | 4/5 [00:42<00:06,  6.66s/it]

{'problem': '有一组程序可以用来计算两个数的最小公倍数（LCM）。该程序使用了最大公约数（GCD）来帮助计算。具体过程如下：\n\n1. 如果两个数相等，则它们的GCD就是其中任意一个数。\n2. 如果第一个数小于第二个数，则用第二个数减去第一个数，并对新结果递归地计算GCD。\n3. 如果第一个数大于第二个数，则交换两个数并递归地计算GCD。\n\n基于此GCD的计算，我们可以通过以下公式计算两个数的LCM：\n\\[ \\text{LCM}(x, y) = \\frac{x}{\\text{GCD}(x, y)} \\times y \\]\n\n根据上述原理，请回答以下选择题：', 'questions': [{'question': '选择题 1：\n已知一个数和9的最小公倍数是9，这个数是多少？', 'options': ['1', '3', '6', '9'], 'answer': 'D'}, {'question': '选择题 2：\n7和2的最小公倍数是8，这个说法正确吗？', 'options': ['正确', '错误', '取决于7和2的最大公约数', '以上都不对'], 'answer': 'B'}, {'question': '选择题 3：\n3和9的最小公倍数是3，这个说法正确吗？', 'options': ['正确', '错误', '取决于3和9的最大公约数', '以上都不对'], 'answer': 'B'}, {'question': '选择题 4：\n3和4的最小公倍数是3，这个说法正确吗？', 'options': ['正确', '错误', '取决于3和4的最大公约数', '以上都不对'], 'answer': 'B'}], 'id': 'round1_train_data_003'}
Error: Expecting value: line 5 column 61 (char 381)


100%|██████████| 5/5 [00:43<00:00,  8.74s/it]

{'problem': '在一场山地自行车比赛中，四位选手分别取得了第一、第二、第三和第四名。不同的颜色代表不同的排名。下面是一些关于这场比赛和选手排名的信息：\n\n1. Alan 名列第一。\n2. 第二名的选手穿红色服装。\n3. John 没有穿黄色服装。\n4. 第四名的选手穿蓝色服装。\n5. Steve 的排名和他的服装色是相同的名次。\n6. Kev 的名次排在 Steve 前面。\n7. 第二名的选手穿的不是青色。\n8. 黄色服穿的选手的成绩排在绿色服穿的选手前面。\n9. 确保每四个参数中的所有元素都不相同和符合排名顺序。\n\n根据上述信息， 回答以下选择题：', 'questions': [{'question': '选择题 1:\n根据比赛结果，排在第二名之后的可能是哪些名次？', 'options': ['第一名 第三名', '第三名 第四名'], 'answer': 'B'}, {'question': '选择题 2:\n第一名是不是在第三名之前？', 'options': ['是', '否'], 'answer': 'A'}, {'question': '选择题 3:\n第一名是不是在第一名之前？', 'options': ['是', '否'], 'answer': 'B'}, {'question': '选择题 4:\n第一名是不是在第四名之前？', 'options': ['是', '否'], 'answer': 'A'}], 'id': 'round1_train_data_002'}
Error: Extra data: line 17 column 1 (char 979)
